In [ ]:
import sys, os
import numpy as np
import xarray as xr
import pandas as pd

import seaborn as sns
from scipy.stats import linregress
import matplotlib.pyplot as plt

# Path to modules
sys.path.append('../modules/')
import globalvars
from wrf_utils import load_preprocessed_WRF_data, subset_wrf_ds

ds_lst = []
# Loop over models and variables
for i, model in enumerate(models):
    for j, varname in enumerate(varnames):
        ax = axes[i, j]

        # --- load data ---
        if varname == "uv":
            load_varname = "uv925"
        else:
            load_varname = varname
        ds = load_preprocessed_WRF_data(model, load_varname, anomaly=False)

        # --- subset to community locations ---
        ds = subset_wrf_ds(ds)

        # --- compute 95th percentile for var for each year ---
        ds = ds.groupby("time.year").quantile(0.95, dim="time").rename(year="time")
        ds_lst = []

ds_merged = xr.merge(ds_lst, )



In [1]:
import pandas as pd

fname = '../figs/trend_lineplot_r2_full.csv'
df = pd.read_csv(fname)
df

,model,variable,location,R2
0,cfsr,ivt,Craig,0.02
1,cfsr,ivt,Hoonah,0.05
2,cfsr,ivt,Kasaan,0.02
3,cfsr,ivt,Klukwan,0.01
4,cfsr,ivt,Skagway,0.05
...,...,...,...,...
85,gfdl,snow,Hoonah,0.00
86,gfdl,snow,Kasaan,0.01
87,gfdl,snow,Klukwan,0.09
88,gfdl,snow,Skagway,0.00


In [2]:
# Create a pivot table with MultiIndex rows and columns
df_pivot_multi = df.pivot_table(index=['model', 'location'], columns='variable', values='R2')
print("\nDataFrame with MultiIndex Rows and Columns (from pivot_table):")
print(df_pivot_multi)


DataFrame with MultiIndex Rows and Columns (from pivot_table):
variable        freezing_level   ivt  pcpt  snow    uv
model location                                        
ccsm  Craig               0.27  0.22  0.00  0.07  0.03
      Hoonah              0.16  0.23  0.08  0.00  0.02
      Kasaan              0.26  0.22  0.00  0.02  0.04
      Klukwan             0.11  0.20  0.04  0.05  0.08
      Skagway             0.12  0.24  0.03  0.13  0.01
      Yakutat             0.14  0.24  0.09  0.01  0.02
cfsr  Craig               0.10  0.02  0.09  0.04  0.00
      Hoonah              0.05  0.05  0.07  0.01  0.03
      Kasaan              0.11  0.02  0.00  0.05  0.00
      Klukwan             0.03  0.01  0.00  0.04  0.00
      Skagway             0.04  0.05  0.02  0.17  0.01
      Yakutat             0.04  0.02  0.06  0.08  0.00
gfdl  Craig               0.48  0.14  0.03  0.00  0.00
      Hoonah              0.64  0.15  0.04  0.00  0.00
      Kasaan              0.49  0.16  0.00  0.01  0.01
 

In [3]:
df_pivot_multi

variable        freezing_level   ivt  pcpt  snow    uv
model location                                        
ccsm  Craig               0.27  0.22  0.00  0.07  0.03
      Hoonah              0.16  0.23  0.08  0.00  0.02
      Kasaan              0.26  0.22  0.00  0.02  0.04
      Klukwan             0.11  0.20  0.04  0.05  0.08
      Skagway             0.12  0.24  0.03  0.13  0.01
      Yakutat             0.14  0.24  0.09  0.01  0.02
cfsr  Craig               0.10  0.02  0.09  0.04  0.00
      Hoonah              0.05  0.05  0.07  0.01  0.03
      Kasaan              0.11  0.02  0.00  0.05  0.00
      Klukwan             0.03  0.01  0.00  0.04  0.00
      Skagway             0.04  0.05  0.02  0.17  0.01
      Yakutat             0.04  0.02  0.06  0.08  0.00
gfdl  Craig               0.48  0.14  0.03  0.00  0.00
      Hoonah              0.64  0.15  0.04  0.00  0.00
      Kasaan              0.49  0.16  0.00  0.01  0.01
      Klukwan             0.62  0.16  0.00  0.09  0.02
      Skagway             0.61  0.12  0.00  0.00  0.02
      Yakutat             0.55  0.11  0.01  0.04  0.00